In [86]:
import pandas as pd

dataset = pd.read_csv("dataset.csv")
alternative = pd.read_csv("alternative.csv")

In [87]:
squared = alternative.applymap(lambda x: x ** 2)
squared.head()

,RAM,Harga,Penyimpanan,Prosesor,Kamera
0,25,4,9,16,16
1,25,4,4,9,16
2,16,16,4,9,9
3,25,1,16,16,16
4,16,9,9,9,16


In [88]:
squared_sum = squared.sum()
squared_sum

RAM            308
Harga          233
Penyimpanan    155
Prosesor       221
Kamera         313
dtype: int64

In [89]:
from math import sqrt

criteria = [x for x in alternative]

def f_n(row):
  result = []
  for i, j in zip(row, squared_sum):
    result.append(i / sqrt(j))
  return result

temp = [x for x in alternative.apply(f_n, axis=1)]
normalization = pd.DataFrame(temp, columns=criteria)
normalization.head()

,RAM,Harga,Penyimpanan,Prosesor,Kamera
0,0.284901,0.131024,0.240966,0.269069,0.226093
1,0.284901,0.131024,0.160644,0.201802,0.226093
2,0.227921,0.262049,0.160644,0.201802,0.169570
3,0.284901,0.065512,0.321288,0.269069,0.226093
4,0.227921,0.196537,0.240966,0.201802,0.226093


In [90]:
weight = pd.read_csv("weight.csv")
weight

,Bobot,Atribut
0,5,benefit
1,3,cost
2,3,benefit
3,5,benefit
4,1,benefit


In [91]:
def f_n_t(row):
  result = []
  for i, j in zip(row, weight['Bobot']):
    result.append(i * float(j))
  return result

temp = [x for x in normalization.apply(f_n_t, axis=1)]
weighted_normalization = pd.DataFrame(temp, columns=criteria)
weighted_normalization.head()

,RAM,Harga,Penyimpanan,Prosesor,Kamera
0,1.424507,0.393073,0.722897,1.345346,0.226093
1,1.424507,0.393073,0.481932,1.009009,0.226093
2,1.139606,0.786146,0.481932,1.009009,0.169570
3,1.424507,0.196537,0.963863,1.345346,0.226093
4,1.139606,0.589610,0.722897,1.009009,0.226093


In [92]:
types = ['positif', 'negatif']

ideal_solution = []
for column, attribute in zip(weighted_normalization, weight['Atribut']):
  val = weighted_normalization[column]

  temp = []
  for i in types:

    result = 0
    if i == 'positif':
      result = min(val) if attribute == 'cost' else max(val)
    else:
      result = max(val) if attribute == 'cost' else min(val)

    temp.append(result)
  ideal_solution.append(temp)

solution = pd.DataFrame(ideal_solution, columns=types)
solution

,positif,negatif
0,1.424507,0.569803
1,0.196537,0.982683
2,0.963863,0.240966
3,1.345346,0.672673
4,0.226093,0.169570


In [93]:
def func(row):
  result = []
  for j in types:

    temp = 0
    for index, ideal in enumerate(solution[j]):
      temp += (row[index] - ideal)**2
    
    result.append(sqrt(temp))
    
  return result

temp = [x for x in weighted_normalization.apply(func, axis=1)]
total = pd.DataFrame(temp, columns=types)
total.head()

,positif,negatif
0,0.310952,1.328947
1,0.619683,1.119170
2,0.881694,0.731087
3,0.000000,1.525388
4,0.637857,0.909811


In [94]:
def fun(row):
  return row[1] / (row[0] + row[1])

preference = total.apply(fun, axis=1)
pd.DataFrame(preference, columns=['Preferensi']).head()

,Preferensi
0,0.810383
1,0.643625
2,0.453309
3,1.000000
4,0.587859


In [98]:
best = []
for index, val in enumerate(preference):
  if val == 1:
    best.append(index)

print("Best alternative:")

column_name = dataset.columns[0]
index = [x+1 for x in best]
names = [dataset[column_name][x] for x in best]
pd.DataFrame(names, index=index, columns=[column_name])

Best alternative:


,Nama
4,Alienware M1
8,Legion Y1
12,ROG Strix 1
17,MSI GF 1
